In [1]:
#Author Arnab Debnath
#Use as you need. Please don't upload your edited version on github though.

In [1]:
import os
import json
import copy

In [2]:
from google_trans_new import google_translator

In [3]:
SQUAD_FILE_PATH = '../Assignment-4/dataset/squad_train-v1.1.json'

In [4]:
squad_dataset = json.load(open(SQUAD_FILE_PATH))

In [5]:
translator = google_translator(timeout=10)
SRC_LANG = 'en'
target_languages = ['ar', 'bn', 'fi', 'id', 'ko', 'ru', 'sw', 'te']
target_languages = ['id']

In [6]:
squad_dataset_copy = copy.deepcopy(squad_dataset)
#data is divided into batches as I am facing timeout issue.
#data_100 = squad_dataset_copy['data'][:100]
#data_200 = squad_dataset_copy['data'][100:200]
#data_300 = squad_dataset_copy['data'][200:300]
#data_400 = squad_dataset_copy['data'][300:400]
#data_500 = squad_dataset_copy['data'][400:]
data = squad_dataset_copy['data']

In [7]:
for target_language in target_languages:
    #data = data_100 #change this for every batch. 
    for index, i in enumerate(data):
        title = i['title']
        try:
            translated_title = translator.translate(title, lang_src=SRC_LANG, lang_tgt=target_language)
        except:
            print("Title", title)
            translated_title = translator.translate(title, lang_src=SRC_LANG, lang_tgt=target_language)
        i['title'] = translated_title
        
        paragraphs = i['paragraphs']
        k = 0
        while k < len(paragraphs):
            paragraph = paragraphs[k]
            context = paragraph['context']
            try:
                translated_context = translator.translate(context, lang_src=SRC_LANG, lang_tgt=target_language)
            except AttributeError as e:
                # One valid paragraph couldn't be translated. Don't know the reason.
                # The specific paragraph was removed later on.
                print(f"index {index} paragraph {k}")
                print("ERROR")
                print("PARAGRAPH SKIPPED")
                k += 1
                continue
            except:
                # Sometimes this happens for no reason. Translating it again solves it.
                print("context")
                translated_context = translator.translate(context, lang_src=SRC_LANG, lang_tgt=target_language)
            
            qas = paragraph['qas']
            j = 0
            flag_qas = False
            while j < len(qas):
                qa = qas[j]
                answer = qa['answers'][0] #For each question, there is only 1 answer.
                question = qa['question']
                try:
                    translated_question = translator.translate(question, lang_src=SRC_LANG, lang_tgt=target_language)
                except:
                    # Sometimes this happens for no reason. Translating it again solves it.
                    print("Question", question)
                    translated_question = translator.translate(question, lang_src=SRC_LANG, lang_tgt=target_language)
                answer_start = answer['answer_start']
                
                text = answer['text']
                try:
                    translated_text = translator.translate(text, lang_src=SRC_LANG, lang_tgt=target_language)
                except:
                    # Sometimes this happens for no reason. Translating it again solves it.
                    print("text", text)
                    translated_text = translator.translate(text, lang_src=SRC_LANG, lang_tgt=target_language)
                
                    
                start_index = translated_context.find(translated_text)
                if start_index != -1:
                    answer['answer_start'] = start_index
                    answer['text'] = translated_text
                    qa['question'] = translated_question  
                    
                    flag_qas = True
                    j += 1
                else:
                    qas.pop(j)
            
            if flag_qas == False:
                paragraphs.pop(k)
            else:
                paragraph['context'] = translated_context
                k += 1
        print(index)
        if len(paragraphs) == 0:
            data.pop(index)
            print("ALL PARAGRAPHS REMOVED.")

Question How many departments are within the Stinson-Remick Hall of Engineering?
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
Question How many people worked for the Bell Boatyard?
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
Question Morningside Park interrupts which street?
94
context
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
text the American Community Survey
128
129
130
131
132
133
134
135
136
137
Question When did the agency acheive a semi-automated air traffic control system?
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
Question Min and Hakka dialects are found 

In [9]:
out_path = '../Assignment-4/dataset/google_translate_train'

In [10]:
out_file = 'squad_id_google.json'
out_file_path = os.path.join(out_path, out_file)
with open(out_file_path, 'w', encoding='utf8') as outfile:
    json.dump(squad_dataset_copy, outfile)